In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point, LineString

from load_data import get_pos, get_spikes
from plotting_functions import plot_intersects, plot_zone

import vdmlab as vdm

In [ ]:
pos = dict()
pos['x'] = np.array([2, 4, 6, 8])
pos['y'] = np.array([7, 5, 3, 1])
pos['time'] = np.array([0., 1., 2., 3.])

binsize = 2
xedges = np.arange(pos['x'].min(), pos['x'].max()+binsize, binsize)
yedges = np.arange(pos['y'].min(), pos['y'].max()+binsize, binsize)

spikes = dict()
spikes['time'] = np.array([[0., 3., 3., 3.],
                           [2.5]])

tuning_curves = vdm.tuning_curve_2d(spikes, pos, xedges, yedges, sampling_rate=1.)

In [ ]:
plt.figure()
xx, yy = np.meshgrid(xedges, yedges)
for tuning_curve in tuning_curves:
    pp = plt.pcolormesh(xx, yy, tuning_curve, cmap='YlGn')
    plt.colorbar(pp)
    plt.axis('off')
    plt.show()

In [ ]:
plt.plot(pos['x'], pos['y'], 'g.', ms=10)
plt.xlim(0.5, 8.5)
plt.ylim(0.5, 7.5)
plt.show()

In [ ]:
one_line = LineString([[2, 2], [2, 4], [2, 6], [2, 8]])

one_start = Point([2, 2])
one_stop = Point([2, 8])

expand_by = 1

one_zone = vdm.expand_line(one_start, one_stop, one_line, expand_by)

In [ ]:
plt.plot(pos['x'], pos['y'], 'g.', ms=10)
plt.plot(one_zone.exterior.xy[0], one_zone.exterior.xy[1], 'b', lw=1)
plt.xlim(-1, 10)
plt.ylim(-1, 10)
plt.show()

In [ ]:
xcenters = np.array((xedges[1:] + xedges[:-1]) / 2.)
ycenters = np.array((yedges[1:] + yedges[:-1]) / 2.)

In [ ]:

tuning_points = []
for i in itertools.product(ycenters, xcenters):
    tuning_points.append(i)
tuning_points = np.array(tuning_points)
tuning_points

In [ ]:
field_thresh = 0
fields_tc = []
for neuron_tc in tuning_curves:
    field_idx = neuron_tc.flatten() > field_thresh
    field = tuning_points[field_idx]
    for pt in field:
        point = Point([pt[0], pt[1]])
        if one_zone.contains(point):
            fields_tc.append(neuron_tc)
fields_tc

In [2]:
def expand_line(start_pt, stop_pt, line, expand_by):
    line_expanded = line.buffer(expand_by)
    zone = start_pt.union(line_expanded).union(stop_pt)
    return zone

def find_zones(info, expand_by=6):
    u_line = LineString(info.u_trajectory)
    shortcut_line = LineString(info.shortcut_trajectory)
    novel_line = LineString(info.novel_trajectory)

    u_start = Point(info.u_trajectory[0])
    u_stop = Point(info.u_trajectory[-1])
    shortcut_start = Point(info.shortcut_trajectory[0])
    shortcut_stop = Point(info.shortcut_trajectory[-1])
    novel_start = Point(info.novel_trajectory[0])
    novel_stop = Point(info.novel_trajectory[-1])
    pedestal_center = Point(info.path_pts['pedestal'][0], info.path_pts['pedestal'][1])
    pedestal = pedestal_center.buffer(expand_by*2.2)

    zone = dict()
    zone['u'] = expand_line(u_start, u_stop, u_line, expand_by)
    zone['shortcut'] = expand_line(shortcut_start, shortcut_stop, shortcut_line, expand_by)
    zone['novel'] = expand_line(novel_start, novel_stop, novel_line, expand_by)
    zone['ushort'] = zone['u'].intersection(zone['shortcut'])
    zone['unovel'] = zone['u'].intersection(zone['novel'])
    zone['uped'] = zone['u'].intersection(pedestal)
    zone['shortped'] = zone['shortcut'].intersection(pedestal)
    zone['novelped'] = zone['novel'].intersection(pedestal)
    zone['pedestal'] = pedestal
    
    return zone

In [3]:
def trajectory_fields(tuning_curves, zone, xedges, yedges, field_thresh):
    
    xcenters = np.array((xedges[1:] + xedges[:-1]) / 2.)
    ycenters = np.array((yedges[1:] + yedges[:-1]) / 2.)
    
    tuning_points = []
    for i in itertools.product(ycenters, xcenters):
        tuning_points.append(i)
    tuning_points = np.array(tuning_points)

    fields_tc = dict(u=[], shortcut=[], novel=[], pedestal=[])
    for neuron_tc in tuning_curves:
        field_idx = neuron_tc.flatten() > field_thresh
        field = tuning_points[field_idx]
        for pt in field:
            point = Point([pt[0], pt[1]])
            if zone['u'].contains(point) or zone['ushort'].contains(point) or zone['unovel'].contains(point):
                if neuron_tc not in fields_tc['u']:
                    fields_tc['u'].append(neuron_tc)
            if zone['shortcut'].contains(point) or zone['shortped'].contains(point):
                fields_tc['shortcut'].append(neuron_tc)
            if zone['novel'].contains(point) or zone['novelped'].contains(point):
                fields_tc['novel'].append(neuron_tc)
            if zone['pedestal'].contains(point):
                fields_tc['pedestal'].append(neuron_tc)
                
    return fields_tc

In [4]:
import sys
sys.path.append('E:\\code\\emi_shortcut\\info')
import info.R063d3_info as r063d3
info = r063d3

In [5]:
pickle_filepath = 'E:\\code\\emi_shortcut\\cache\\pickled'

In [6]:
pos = get_pos(info.pos_mat, info.pxl_to_cm)
spikes = get_spikes(info.spike_mat)

In [7]:
import os
import pickle

tc_filename = info.session_id + '_tuning_2d.pkl'
pickled_tc = os.path.join(pickle_filepath, tc_filename)
if os.path.isfile(pickled_tc):
    with open(pickled_tc, 'rb') as fileobj:
        tuning_curves = pickle.load(fileobj)
else:
    binsize = 3
    xedges = np.arange(pos['x'].min(), pos['x'].max()+binsize, binsize)
    yedges = np.arange(pos['y'].min(), pos['y'].max()+binsize, binsize)

    speed = vdm.get_speed(pos)
    t_run = speed['time'][speed['smoothed'] >= info.run_threshold]
    run_idx = np.zeros(pos['time'].shape, dtype=bool)
    for idx in t_run:
        run_idx |= (pos['time'] == idx)

    run_pos = dict()
    run_pos['x'] = pos['x'][run_idx]
    run_pos['y'] = pos['y'][run_idx]
    run_pos['time'] = pos['time'][run_idx]

    t_start = info.task_times['phase3'][0]
    t_stop = info.task_times['phase3'][1]

    t_start_idx = vdm.find_nearest_idx(run_pos['time'], t_start)
    t_stop_idx = vdm.find_nearest_idx(run_pos['time'], t_stop)

    sliced_pos = dict()
    sliced_pos['x'] = run_pos['x'][t_start_idx:t_stop_idx]
    sliced_pos['y'] = run_pos['y'][t_start_idx:t_stop_idx]
    sliced_pos['time'] = run_pos['time'][t_start_idx:t_stop_idx]

    sliced_spikes = dict()
    sliced_spikes['time'] = vdm.time_slice(spikes['time'], t_start, t_stop)
    sliced_spikes['label'] = spikes['label']
    
    tuning_curves = vdm.tuning_curve_2d(sliced_spikes, sliced_pos, xedges, yedges, gaussian_sigma=0.2)

In [8]:
zones = find_zones(info)

In [9]:
binsize = 3
xedges = np.arange(pos['x'].min(), pos['x'].max()+binsize, binsize)
yedges = np.arange(pos['y'].min(), pos['y'].max()+binsize, binsize)

fields_tc = trajectory_fields(tuning_curves, zones, xedges, yedges, field_thresh=5)

NameError: name 'itertools' is not defined

In [ ]:
print(fields_tc['novel'])